In [1]:
import json
from sklearn.externals import joblib
import random
import numpy as np

In [2]:
events_num, weibos_num, event_weibos_num_list, global_sampling_factor, event_sampling_factor_list = joblib.load(
        'file/pkl/event_sampling_factor.pkl')
sampling_num_of_event, certify_num_of_event = joblib.load('file/pkl/certify_num_of_every_event.pkl')
event_features_list = joblib.load('file/pkl/event_features.pkl')

In [4]:
events_num, weibos_num

(5979, 38180)

In [7]:
sum(sampling_num_of_event)

6753

In [9]:
sum([sum(x) for x in certify_num_of_event])

6753

# 计算参数

In [2]:
# （5942）4439:1284:219 = 20.3 : 5.9 : 1
                
# Rumor Features
rumor_weibo_sum = 7880
rumor_certify = 5942
rumor_0 = 4439
rumor_1 = 1284
rumor_2 = 219

In [117]:
def get_event_certify_num(threshold=0.75):
    events_num, weibos_num, event_weibos_num_list, global_sampling_factor, event_sampling_factor_list = joblib.load(
        'file/pkl/event_sampling_factor.pkl')
    # 计算在每个事件中抽取的微博数
    raw_sampling_num_of_event = [weibos_num * global_sampling_factor * factor for factor in event_sampling_factor_list]
    raw_sampling_num_of_event = [int(x + 0.5) for x in raw_sampling_num_of_event]
    # sampling_num_of_event = [1 if x == 0 else x for x in sampling_num_of_event]

    # 以一定的机率更新sampling为0的事件（用来调节sampling的样本数）
    sampling_num_of_event = []
    for sampling_num in raw_sampling_num_of_event:
        rand = random.random()
        if rand > threshold:
            sampling_num = 1 if sampling_num == 0 else sampling_num
        sampling_num_of_event.append(sampling_num)

    # 计算不同的userCertify字段应该分别抽取多少微博
    float_certify_num_of_event = [
        [rumor_0 / rumor_certify * x, rumor_1 / rumor_certify * x, rumor_2 / rumor_certify * x]
        for x in sampling_num_of_event]
    certify_num_of_event = []
    for index, float_certify_num in enumerate(float_certify_num_of_event):
        # the last one
        if index == len(float_certify_num_of_event) - 1:
            certify_num_of_event.append([int(x + 1) for x in float_certify_num])
            continue

        next_float_certify_num = float_certify_num_of_event[index + 1]

        certify_num = [int(x) for x in float_certify_num]
        next_float_certify_num[0] += float_certify_num[0] - certify_num[0]
        next_float_certify_num[1] += float_certify_num[1] - certify_num[1]
        next_float_certify_num[2] += float_certify_num[2] - certify_num[2]

        certify_num_of_event.append(certify_num)

    # 更新在每个事件中抽取的微博数
    sampling_num_of_event = [sum(x) for x in certify_num_of_event]

    joblib.dump((sampling_num_of_event, certify_num_of_event), 'file/pkl/certify_num_of_every_event.pkl')

    # 打印sampling的结果
    c0 = sum([x[0] for x in certify_num_of_event])
    c1 = sum([x[1] for x in certify_num_of_event])
    c2 = sum([x[2] for x in certify_num_of_event])
    print('采样数：{}，userCertify的分布为：({}) {}:{}:{} = {:.1f} : {:.1f} : 1'.format(
        sum(sampling_num_of_event), c0 + c1 + c2, c0, c1, c2, c0 / c2, c1 / c2))
    
get_event_certify_num()

采样数：7972，userCertify的分布为：(7972) 5955:1723:294 = 20.3 : 5.9 : 1


In [118]:
sampling_num_of_event, certify_num_of_event = joblib.load('file/pkl/certify_num_of_every_event.pkl')

In [119]:
sum(sampling_num_of_event)

7972

In [120]:
certify_num_of_event[:5]

[[0, 0, 0], [4, 1, 0], [0, 0, 0], [0, 0, 0], [2, 0, 0]]

In [121]:
c0 = sum([x[0] for x in certify_num_of_event])
c1 = sum([x[1] for x in certify_num_of_event])
c2 = sum([x[2] for x in certify_num_of_event])

print('({}) {}:{}:{} = {:.1f} : {:.1f} : 1'.format(c0+c1+c2, c0, c1,c2, c0/c2, c1/c2))

(7972) 5955:1723:294 = 20.3 : 5.9 : 1


# 查看result

In [42]:
result_certify_num_of_event, result_index_of_event = joblib.load('file/pkl/result.pkl')

In [44]:
result_certify_num_of_event[:5]

[[2, 0, 0], [3, 1, 0], [0, 0, 0], [0, 0, 0], [2, 1, 0]]

In [346]:
float_certify_num_of_event = [[1.5,1.4,5.6],[2.4,3.5,8.8]]
# int_certify_num_of_event = [int(x + 0.5) for float_num in float_certify_num_of_event for x in float_num]
test = [[int(a) for a in x] for x in float_certify_num_of_event]

In [347]:
test

[[1, 1, 5], [2, 3, 8]]

In [342]:
b

[[100, 200], [200, 400], [300, 600]]

In [337]:
e

3